In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv("IMDB Dataset.csv")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
# Preprocessing of data:
# Convert all to lower cases 
# Remove special characters from the comments.
# Check and remove null values

data['review'] = data['review'].str.lower()
data['review'] = data['review'].replace(r'[^0-9a-z\s]', '', regex= True)
print(data['review'])

# do the same with the sentiments:
data['sentiment'] = data['sentiment'].str.lower()
data['sentiment'] = data['sentiment'].replace(r'[^0-9a-z\s]', '', regex= True)
data['sentiment']
print(data['sentiment'])

data.shape



0        one of the other reviewers has mentioned that ...
1        a wonderful little production br br the filmin...
2        i thought this was a wonderful way to spend ti...
3        basically theres a family where a little boy j...
4        petter matteis love in the time of money is a ...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    i am a catholic taught in parochial elementary...
49998    im going to have to disagree with the previous...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object
0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object


(50000, 2)

In [4]:
data['sentiment_num'] = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
data['sentiment_num']

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment_num, Length: 50000, dtype: int64

In [5]:
# Time to convert the text data to numbers using 
# Tokenizer: to break sentences as token and 
# Padding: to align input vector in same size 



max_token = 5000 #Sets the maximum number of words to keep in the tokenizer
max_token_len = 200 # Defines the fixed length for each input sequence after padding

tokenizer = Tokenizer(num_words = max_token)
tokenizer.fit_on_texts(data['review'])
tokenized_features = tokenizer.texts_to_sequences(data['review'])



X = pad_sequences(tokenized_features, maxlen = max_token_len)
y = data['sentiment_num']

In [6]:
# time to perform train and test split since data is there in X and y now

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Simple feed forward netwrok: (DNN)

In [ ]:
# Model architecting:
# 1. Model architure
# 2. Model complie
# 3. Model train
# 4. Model evaluate
# 5. Model prediction
''' 
# \\results are following: 
# Test loss: 0.26617515087127686
# Test accuracy: 0.8896999955177307

"""Review: Good music, Good graphics, Ok Ok story line and actions too voilence for kids.
Sentiment: Negative
Confidence: 0.6246

Review: Greate music graphics, Super story line and actions. was easy to understand and enjoyed alot
Sentiment: Positive
Confidence: 0.8366"""

 
------------------------------------------------------------
Classification Report:
------------------------------------------------------------
              precision    recall  f1-score   support

    Negative       0.89      0.89      0.89      4961
    Positive       0.89      0.89      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

'''


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=max_token, output_dim=128, input_length=max_token_len))
model.add(tf.keras.layers.GlobalAveragePooling1D())  # Converts 3D output to 2D for Dense layer
model.add(tf.keras.layers.Dense(units= 128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units= 64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))

model.summary()

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

LSTM Model:

In [ ]:
'''
Test loss: 0.6931105852127075
Test accuracy: 0.503600001335144

Review: Good music, Good graphics, Ok Ok story line and actions too voilence for kids
Sentiment: Positive
Confidence: 0.5100

Review: Greate music graphics, Super story line and actions. was easy to understand and enjoyed alot
Sentiment: Positive
Confidence: 0.7052

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
              precision    recall  f1-score   support

    Negative       0.87      0.92      0.89      4961
    Positive       0.91      0.87      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000


'''


model1 = tf.keras.models.Sequential()

model1.add(tf.keras.layers.Embedding(input_dim = max_token, output_dim=128))
model1.add(tf.keras.layers.LSTM(units = 128, return_sequences=False))
model1.add(tf.keras.layers.Dense(units = 128, activation= 'relu'))
model1.add(tf.keras.layers.Dropout(0.2))
model1.add(tf.keras.layers.Dense(units= 64, activation='relu'))
model1.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
model1.summary()



Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

BiDirectional LSTM

In [9]:
model2 = tf.keras.models.Sequential()

model2.add(tf.keras.layers.Embedding(input_dim = max_token, output_dim=128))
model2.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 128, return_sequences=False)))
model2.add(tf.keras.layers.Dense(units = 128, activation= 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(units= 64, activation='relu'))
model2.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Transformer

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att= (tf.keras.layers.MultiHeadAttention(num_heads = num_heads, key_dim=emb_dia))
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim),
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

print("TransformerBlock class created successfully!")
    
        
emb_dia = 32
no_of_heads = 2
dropout_rate= 0.2
feed_forward_units = 32

transformer = tf.keras.models.Sequential()

transformer.add(tf.keras.layers.Embedding(input_dim= max_token, output_dim= emb_dia, input_length = max_token_len))
# transformer.add(TransformerBlock(emb_dia, num_heads=no_of_heads, ff_dim=feed_forward_units))
transformer.add(TransformerBlock(emb_dia, num_heads=no_of_heads, ff_dim=feed_forward_units))
transformer.add(tf.keras.layers.GlobalAveragePooling1D())
transformer.add(tf.keras.layers.Dropout(0.2))
transformer.add(tf.keras.layers.Dense(32, activation='relu'))
transformer.add(tf.keras.layers.Dropout(0.2))
transformer.add(tf.keras.layers.Dense(1, activation='sigmoid'))

transformer.summary()

TransformerBlock class created successfully!


c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_8             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile:
# model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'] )

# model1.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'] )

# model2.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'] )

transformer.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'] )

In [22]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2,min_lr=0.0001, verbose=1)
]

# history= model.fit(X_train, y_train, epochs= 10, batch_size= 32, callbacks= callbacks, validation_data = (X_test, y_test))

# history1= model1.fit(X_train, y_train, epochs= 10, batch_size= 32, callbacks= callbacks, validation_data = (X_test, y_test))

# history2= model2.fit(X_train, y_train, epochs= 10, batch_size= 32, callbacks= callbacks, validation_data = (X_test, y_test))

transformer_history = transformer.fit(X_train, y_train, epochs= 10, batch_size= 32, callbacks= callbacks, validation_data = (X_test, y_test))

# score= model.evaluate(X_test, y_test, verbose=1)
# score= model1.evaluate(X_test, y_test, verbose=1)


score= transformer.evaluate(X_test, y_test, verbose=1)


print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sequential_10_1/embedding_9_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\thedo\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\thedo\AppData\Roaming\Python\Python313\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\thedo\AppData\Roaming\Python\Python313\site-packages\ipykernel\kernelapp.py", line 758, in start

  File "C:\Users\thedo\AppData\Roaming\Python\Python313\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\thedo\anaconda3\Lib\asyncio\base_events.py", line 683, in run_forever

  File "c:\Users\thedo\anaconda3\Lib\asyncio\base_events.py", line 2042, in _run_once

  File "c:\Users\thedo\anaconda3\Lib\asyncio\events.py", line 89, in _run

  File "C:\Users\th

In [ ]:
# Making predictions on new reviews
new_reviews = [
    "Good music, Good graphics, Ok Ok story line and actions too voilence for kids",
    "Greate music graphics, Super story line and actions. was easy to understand and enjoyed alot",
    "The movie is too violent to watch with kids under five",
    'Not that good'
]

# Preprocess the new reviews
new_reviews_lower = [review.lower() for review in new_reviews]
import re
new_reviews_clean = [re.sub(r'[^0-9a-z\s]', '', review) for review in new_reviews_lower]

# Tokenize and pad the new reviews
new_reviews_tokenized = tokenizer.texts_to_sequences(new_reviews_clean)
new_reviews_padded = pad_sequences(new_reviews_tokenized, maxlen=max_token_len)

# Make predictions
# predictions = model.predict(new_reviews_padded, verbose=0)

predictions = model2.predict(new_reviews_padded, verbose=0)

# Display results
for i, review in enumerate(new_reviews):
    sentiment = "Positive" if predictions[i][0] > 0.5 else "Negative"
    confidence = predictions[i][0] if predictions[i][0] > 0.5 else 1 - predictions[i][0]
    print(f"\nReview: {review}")
    print(f"Sentiment: {sentiment}")
    print(f"Confidence: {confidence:.4f}")


Review: Good music, Good graphics, Ok Ok story line and actions too voilence for kids
Sentiment: Negative
Confidence: 0.5175

Review: Greate music graphics, Super story line and actions. was easy to understand and enjoyed alot
Sentiment: Positive
Confidence: 0.8368

Review: The movie is too violent to watch with kids under 5
Sentiment: Positive
Confidence: 0.7652

Review: Not that good
Sentiment: Negative
Confidence: 0.7761


In [13]:
# Model evaluation:
# Essential: Classification Report 
# Steps: 
# 1. Make prediction on test data
# 2. Give this prediction and real test values to the method
# 3. Generate the metrics, interpret it


#1. Make prediction and get te value of prediction:




# y_predition_probability = model.predict(X_test, verbose=0)
y_predition_probability = model2.predict(X_test, verbose=0)

y_prediction = (y_predition_probability > 0.5).astype(int).flatten()

# Classification Report:
print('-' * 60)
print('Classification Report:')
print('-' * 60)
print(classification_report(y_test, y_prediction, target_names= ['Negative', 'Positive']))

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
              precision    recall  f1-score   support

    Negative       0.90      0.89      0.89      4961
    Positive       0.89      0.90      0.90      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



In [ ]:
'''
┌─────────────────────────────────────────────────────────┐
│                    MODEL COMPARISON                      │
├─────────────────────────────────────────────────────────┤
│                                                          │
│  DNN (Pooling)              LSTM                        │
│  ├─ Accuracy: 89.0%        ├─ Accuracy: 89.2% ⭐      │
│  ├─ Speed: ⚡⚡⚡⚡⚡          ├─ Speed: ⚡⚡              │
│  ├─ Balanced: ✅✅          ├─ Context: ✅✅✅          │
│  └─ Simple: ✅✅✅           └─ Complex: ✅             │
│                                                          │
│  Winner: Speed            Winner: Accuracy              │
│                                                          │
└─────────────────────────────────────────────────────────┘
'''